In [1]:
import spacy
from geopy.geocoders import Nominatim
from geojson import FeatureCollection
import json

In [3]:
sentence = "Tra i monumenti di Torino più noti anche all'estero sono da citare l'ottocentesca Mole Antonelliana, \
    simbolo incontrastato della città, che ospita il Museo nazionale del Cinema; il Palazzo Reale (antica dimora dei duchi ed in seguito dei re di Casa Savoia);"

nlp = spacy.load('it_core_news_sm', exclude=["ner"])
ner_ita = spacy.load('it_nerIta_trf')

nlp.add_pipe("transformer", name="trf_ita", source=ner_ita, last=True)
nlp.add_pipe("ner", name="ner_ita", source=ner_ita, last=True)

doc = nlp(sentence)

locs = []
labels = ['LOC', 'FAC']
for ent in doc.ents:
    if ent.label_ in labels :
        locs.append(ent.text)

locs

['Mole Antonelliana', 'il Museo nazionale del Cinema', 'il Palazzo Reale']

In [4]:
geocoder = Nominatim(user_agent="myGeocoder")

sents = list(doc.sents)
features = []

for loc in locs:
    sent = [sentence.text for sentence in sents if loc in sentence.text][0]
    print(sent)
    loc = loc.replace("il ", "")
    print(loc)
    res = geocoder.geocode({"street": loc, "city": "Torino", "state": "Piemonte"}, language='it', exactly_one=True)
    print((res.longitude, res.latitude))

    loc_feature = Feature(
        geometry=Point((res.longitude, res.latitude)),
        properties={
            "entity": loc,
            "name_location": res.address,
            "class": res.raw['class'],
            "type": res.raw['type'],
            "snippet": sent
        }
    )
    features.append(loc_feature)



Tra i monumenti di Torino più noti anche all'estero sono da citare l'ottocentesca Mole Antonelliana,     simbolo incontrastato della città, che ospita il Museo nazionale del Cinema;
Mole Antonelliana
(7.693191847163605, 45.06898875)
Tra i monumenti di Torino più noti anche all'estero sono da citare l'ottocentesca Mole Antonelliana,     simbolo incontrastato della città, che ospita il Museo nazionale del Cinema;
Museo nazionale del Cinema
(7.6929702, 45.0691058)
il Palazzo Reale (antica dimora dei duchi ed in seguito dei re di Casa Savoia);
Palazzo Reale
(7.686963196696155, 45.073053349999995)


In [5]:
fc = FeatureCollection(features)

with open('res/paper.geojson', 'w', encoding='utf-8') as f:
    json.dump(fc, f, ensure_ascii=False, indent=4)
    print("The result complete has been saved as a file inside the response folder")

The result complete has been saved as a file inside the response folder
